# Marketing Budget Allocation

Goal - Use linear programming to build a simple marketing budget allocation strategy.

In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp

**Creating general function for optimizing (Max or Min) based on constraint equations**

In [15]:
def create_model(A, sense, b, obj, opt=gp.GRB.MAXIMIZE):
    # creating model
    model = gp.Model()

    # creating variable an setting the constraints
    modx = model.addMVar(A.shape[1])
    mod_con = model.addMConstrs(A, modx, sense, b)

    # setting the objective function
    model.setMObjective(None, obj, 0, sense=opt)

    # restricting gurobi logs
    model.Params.OutputFlag = 0

    # optimizing the function
    model.optimize()
    
    return model

## ROI optimization using first estimates

Assume that your company is deciding how to spend a marketing budget of $10M.  You work in the marketing department as a data scientist and the chief marketing officer has asked you write a report recommending how to spread this budget among several marketing mediums.  Your department has employed an outside consulting firm to estimate the return on investment (ROI) of each marketing medium under consideration.  The results are in the table below, and also in a CSV attached to this assignment:

In [20]:
# reading the ROI data from two different firms
roi_df = pd.read_csv('ROI_data.csv')

# creating numpy array for model usage
roi = roi_df[roi_df['Platform'] == 'ROI'].values[0, 1:]
second_roi = roi_df[roi_df['Platform'] != 'ROI'].values[0, 1:]

# variable for total budget
total_budget = 10000000

roi_df

,Platform,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,ROI,0.031,0.049,0.024,0.039,0.016,0.024,0.046,0.026,0.033,0.044
1,Second Firms ROI Estimate,0.049,0.023,0.024,0.039,0.044,0.046,0.026,0.019,0.037,0.026


On top of these ROIs, your boss has decided to constrain your budget as follows:
1.	The amount invested in print and TV should be no more than the amount spent on Facebook and Email. Surprisingly, email seems to be a great channel for reaching real people.
2.	The total amount used in social media (Facebook, LinkedIn, Instagram, Snapchat, and Twitter) should be at least twice of SEO and AdWords.
3.	For each platform, the amount invested should be no more than $3M.


Assuming variables $x_0$ to $x_{9}$ to be the fraction of total budget for different platforms - from Print to Email in the above csv

In [24]:
# initializing constraint matrix and constraint limit and directions
A = np.zeros((13, 10))
b = np.zeros(13)
sense = ['']*13

# Constraint 1:
# total_budget*(x_0 + x_1) - total_budget*(x_4 + x_9) <= 0
A[0, 0] = 1
A[0, 1] = 1
A[0, 4] = -1
A[0, 9] = -1
b[0] = 0
sense[0] = '<'

# Constraint 2:
# total_budget*(x_4 + x_5 + x_6 + x_7 + x_8) - 2*total_budget*(x_2 + x_3) >= 0
A[1, 4:9] = 1
A[1, 2:4] = -2
b[1] = 0
sense[1] = '>'

# Constraint 3 to 12:
# x_i <= 0.3
for i in range(10):
    A[i+2, i] = 1
    b[i+2] = 0.3
    sense[i+2] = '<'

# Constraint 13:
# sum of all x_i <= 1
A[12, :] = 1
b[12] = 1
sense[12] = '<'

A, sense, b

(array([[ 1.,  1.,  0.,  0., -1.,  0.,  0.,  0.,  0., -1.],
        [ 0.,  0., -2., -2.,  1.,  1.,  1.,  1.,  1.,  0.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]]),
 ['<', '>', '<', '<', '<', '<', '<', '<', '<', '<', '<', '<', '<'],
 array([0. , 0. , 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 1. ]))

Formulate the marketing budget allocation problem as a linear program.  Use gurobi to find the optimal budget allocation.

In [74]:
# creating advertising model for first estimates
adv_model = create_model(A, sense, b, obj=roi, opt=gp.GRB.MAXIMIZE)

<ipython-input-74-038dee29a028>:2: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  adv_model = create_model(A, sense, b, obj=roi, opt=gp.GRB.MAXIMIZE)


In [75]:
# creating an output df with different firms ROI for Platform allocation
output_df = roi_df.T.reset_index()
output_df.columns = output_df.iloc[0]
output_df = output_df.iloc[1:]

# creating new column in output df for budget allocation
output_df['Allocation'] = adv_model.x

**Platform wise allocation based on ROI maximisation for the marketing budget is below**

In [76]:
output_df[['Platform', 'Allocation']]

,Platform,Allocation
1,Print,0.0
2,TV,0.3
3,SEO,0.0
4,AdWords,0.1
5,Facebook,0.0
6,LinkedIn,0.0
7,Instagram,0.3
8,Snapchat,0.0
9,Twitter,0.0
10,Email,0.3


In [77]:
print('Total ROI from ${} marketing budget is {} dollars which is {}%'.format(
    total_budget, round(adv_model.objVal*total_budget), round(adv_model.objVal*100,3)))

Total ROI from $10000000 marketing budget is 456000 dollars which is 4.56%


Your boss is happy to see the promising results presented by the marketing department. However, your boss is also very concerned because your boss recalls being somewhat disappointed after following such recommendations in the past. To be cautious about the decision, your team has decided to get another opinion about the ROI data and rerun the analysis.  The second consulting firm returns the estimates of the ROI data in the table below (also in the CSV file mentioned above).  You are asked to compare the two optimal allocations from these two ROI estimates.  

## ROI optimization using second firm's estimates

In [78]:
# creating advertising model for second estimates
adv_model2 = create_model(A, sense, b, obj=second_roi, opt=gp.GRB.MAXIMIZE)

<ipython-input-78-3f7373cb1b73>:2: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  adv_model2 = create_model(A, sense, b, obj=second_roi, opt=gp.GRB.MAXIMIZE)


In [79]:
# creating new column in output df for budget allocation
output_df['Second Allocation'] = adv_model2.x

**Platform wise allocation based on ROI maximisation using second ROI for the marketing budget is below**

In [80]:
output_df[['Platform', 'Second Allocation']]

,Platform,Second Allocation
1,Print,0.3
2,TV,0.0
3,SEO,0.0
4,AdWords,0.1
5,Facebook,0.3
6,LinkedIn,0.3
7,Instagram,0.0
8,Snapchat,0.0
9,Twitter,0.0
10,Email,0.0


In [81]:
print('For Second firm"s ROI estimates: Total ROI from ${} marketing budget is {} dollars which is {}% '.format(
    total_budget, round(adv_model2.objVal*total_budget), round(adv_model2.objVal*100,3)))

For Second firm"s ROI estimates: Total ROI from $10000000 marketing budget is 456000 dollars which is 4.56% 


## Budget allocation and optimal ROI for First firm's vs Second firm's estimates

In [85]:
if sum(output_df['ROI'] * output_df['Allocation']) == sum(
    output_df['Second Firms ROI Estimate'] * output_df['Second Allocation']):
    print('Maximum ROI obtainied using optimized allocation for both firm"s estimates are same')

Maximum ROI obtainied using optimized allocation for both firm"s estimates are same


Are the allocations the same?  Assuming the first ROI data is correct, if you were to use the second allocation (the allocation that assumed the second ROI data was correct) how much lower would the objective be relative to the optimal objective (the one that uses the first ROI data and the first allocation)?  Assuming the second ROI data is correct, if you used the first allocation how much lower would the objective be relative to the optimal objective?  Do you think the third constraint above, based on your boss’ experience, is useful?